In [56]:
from agent.scrape import scrape_sklearn_website

In [57]:
# sklearn_data = scrape_sklearn_website()

In [58]:
import json

with open("data/sklearn_function_openai.json","r") as jsonfile:
    sklearn_data = json.load(jsonfile)

In [59]:
from agent.utils import build_no_summary_graph,get_parents_dict

In [60]:
sklearn_graph = build_no_summary_graph(sklearn_data)

In [61]:
parents_dict = get_parents_dict(sklearn_graph)

In [47]:
parents_dict['sklearn.model_selection']['sklearn.model_selection#Splitters']

['K-fold iterator variant with non-overlapping groups. Each group will appear exactly once in the test set across all folds (the\nnumber of distinct groups has to be at least equal to the number of folds). The folds are approximately balanced in the sense that the number of\nsamples is approximately the same in each test fold. Read more in the User Guide. For visualisation of cross-validation behaviour and\ncomparison between common scikit-learn split methods\nrefer to Visualizing cross-validation behavior in scikit-learn Number of folds. Must be at least 2. Changed in version 0.22: n_splits default value changed from 3 to 5. See also For splitting the data according to explicit domain-specific stratification of the dataset. Takes class information into account to avoid building folds with imbalanced class proportions (for binary or multiclass classification tasks). Notes Groups appear in an arbitrary order throughout the folds.  Get metadata routing of this object. Please check User G

## LCEL TO SUMMARIZE THE DATA

In [8]:
# ans = summary_chain.invoke({"descriptions":"\n\n".join(parents_dict['sklearn']['sklearn#defaults'])})

In [9]:
# ans = await summary_chain.abatch([{"descriptions":"\n\n".join(parents_dict['sklearn']['sklearn#defaults'])}],config={"max_concurrency": 5})
# ans

In [8]:
from typing import List
def split_description(parent_text:List[str],MAX_WORDS:int=500):
        split_s = []
        running_num_words = 0
        curr_func_string = ""
        for txt in parent_text:
            num_words = len(txt.split(" "))
            running_num_words += num_words
            if running_num_words > MAX_WORDS:
                running_num_words = num_words
                split_s.append(curr_func_string)
                curr_func_string = txt
            else:
                curr_func_string += txt + "\n"
        if split_s == []:
            split_s.append(curr_func_string)
        split_s = [s for s in split_s if s!=""]
        return split_s

In [9]:
parent_child_dict = {pn:{} for pn in parents_dict}

for parent_name,child_nodes in parents_dict.items():
    for child_name,child_texts in child_nodes.items():
        child_split_list = split_description(child_texts,500)
        parent_child_dict[parent_name].update({child_name:child_split_list})        

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv,find_dotenv
# from langchain_openai_limiter import LimitAwaitChatOpenAI, ChooseKeyChatOpenAI

load_dotenv(find_dotenv(),override=True)
# chat_model_limit_await = LimitAwaitChatOpenAI(
#     chat_openai=model,
#     limit_await_timeout=60.0,
#     limit_await_sleep=0.1,
# )
# chat_model_key_choose = ChooseKeyChatOpenAI(
#     chat_openai=chat_model_limit_await,
#     openai_api_keys=[
#         os.environ["OPENAI_API_KEY0"],
#         os.environ["OPENAI_API_KEY1"],
#     ]
# )

summary_prompt = ChatPromptTemplate.from_template(
"""
You are given a list of descriptions of different functions separated by new paragraph.
Your task is to summarize all the text into coherent and detailed summary that covers all the functions descriptions.
Be very diligent and make sure that no function description is left out of the final summary. 

Follow the following format

List of function descriptions: list of function descriptions to be summarized
Summary: summary of all the function descriptions

-----

List of functions descriptions: {descriptions}

Summary: 
"""
)

parent_summary_dict = {pn:{} for pn in parents_dict}

batch_list = []
for parent_name,child_node in parent_child_dict.items():
    for child_name,child_texts in child_node.items():
        for ctext in child_texts:
            batch_list.append({f"{parent_name}-->{child_name}":ctext})


description_list = [{"descriptions":list(desc.values())[0]} for desc in batch_list]

BATCH_SIZE = 5
start_key = '0'
summaries_list = []
for start in tqdm(range(0,len(description_list),BATCH_SIZE),desc="Embedding rows"):
    end = min(start+BATCH_SIZE,len(description_list))
    model = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key=os.environ[f'OPENAI_API_KEY{start_key}'],max_retries=5)
    summary_chain = summary_prompt | model | StrOutputParser()
    curr_description_list = description_list[start:end]
    curr_summary_list = summary_chain.batch(curr_description_list,config={"max_concurrency":5})
    summaries_list.extend(curr_summary_list)



summary_list = summary_chain.batch(description_list,config={"max_concurrency":5})
from tqdm import tqdm

pbar = tqdm(total=len(batch_list))
for sl in summary_list:
    bl = batch_list[sl[0]]
    parent_name = list(bl.keys())[0].split("-->")[0]
    child_name = list(bl.keys())[0].split("-->")[1]
    if child_name in parent_summary_dict[parent_name]:
        parent_summary_dict[parent_name][child_name].append(sl[1])
    else:
        parent_summary_dict[parent_name].update({child_name:[sl[1]]})
    pbar.update(1)

In [20]:
len(description_list),len(batch_list)

(353, 353)

In [18]:
ans = summary_chain.batch_as_completed(description_list,config={"max_concurrency":10})

In [14]:
for i in ans:
    print(i)

(0, "The global scikit-learn configuration offers various options for customization. \n- The configuration includes settings for validation of finiteness, size of temporary arrays, printing of non-default parameters, display of estimators, row vectors per chunk, accelerated pairwise-distances reduction backend, Array API dispatching, output format of transform and fit_transform, metadata routing, validation of hyper-parameters' types and values, and retrieval of current configuration values.\n- Users can choose to skip validation for finiteness for faster processing, set a limit for temporary array size, control the printing of non-default parameters, display estimators as diagrams or text, adjust the number of row vectors per chunk, use the accelerated pairwise-distances reduction backend, enable Array API dispatching, configure output formats, enable metadata routing, and disable validation of hyper-parameters' types and values.\n- These configurations can be customized globally and 

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key=os.environ['OPENAI_API_KEY'],max_retries=5,model_kwargs={"batch_size":20})

In [ ]:
model.model_kwargs

{'batch_size': 20}

In [62]:
# def build_graph(sklearn_graph):
from copy import deepcopy
import re
embed_docs = []
embed_metadata = []

with open("parent_summary.json","r") as jsonfile:
    parent_summary_dict = json.load(jsonfile)

def clean_text(s:str):
    s = re.sub("\n"," ",s)
    s = json.dumps(s)
    return s
for node,attr in sklearn_graph.nodes(data=True):
    type = attr.get('type')
    
    if type == 'function_node':
        ftext = attr['function_text']
        if ftext == "": continue
        embed_docs.append(clean_text(attr['function_text']))
        for k,v in attr.items():
            if not isinstance(v,str):
                attr[k] = str(v)
        embed_metadata.append(attr)
    elif type == 'parent_node':
        embed_docs.append(clean_text(parent_summary_dict[node]))
        for k,v in attr.items():
            if not isinstance(v,str):
                attr[k] = str(v)
        embed_metadata.append(attr)
    elif type == 'sub_level_node':
        for ctext in attr['child_texts']:
            embed_docs.append(clean_text(ctext))
            sub_level_attr = deepcopy(attr)
            del sub_level_attr['child_texts']
            for k,v in sub_level_attr.items():
                if not isinstance(v,str):
                    sub_level_attr[k] = str(v)
            embed_metadata.append(sub_level_attr)

In [63]:
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
from chromadb.utils.batch_utils import create_batches

def build_database(docs, metadata, api_key):
    database_path = "Sklearn"
    collection_name = "sklearn_docs"
    load_dotenv(find_dotenv(), override=True)
    emb_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=api_key, model_name="text-embedding-3-small"
    )

    client = chromadb.PersistentClient(path=database_path)
    sklearn_collection = client.get_or_create_collection(
        name=collection_name, embedding_function=emb_fn
    )

    sklearn_ids = [f"id{i}" for i in range(len(docs))]
    batches = create_batches(
        api=client, ids=sklearn_ids, documents=docs, metadatas=metadata
    )
    for batch in batches:
        sklearn_collection.add(ids=batch[0], documents=batch[3], metadatas=batch[2])
    return sklearn_collection

In [64]:
len(embed_docs[:len(embed_docs)//2]) + len(embed_docs[len(embed_docs)//2:])

1154

In [65]:
from dotenv import load_dotenv,find_dotenv
import os
import time
load_dotenv(find_dotenv(),override=True)
BATCH_SIZE = 200
for start in range(0,len(embed_docs),BATCH_SIZE):
    end = min(start+BATCH_SIZE,len(embed_docs))
    curr_docs = embed_docs[start:end]
    curr_metadata = embed_metadata[start:end]
    sklearn_collection = build_database(curr_docs[:len(curr_docs)//2],curr_metadata[:len(curr_metadata)//2],os.environ['OPENAI_API_KEY0'])
    sklearn_collection = build_database(curr_docs[len(curr_docs)//2:],curr_metadata[len(curr_metadata)//2:],os.environ['OPENAI_API_KEY1'])
    print("Sleeping...")
    time.sleep(60)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 8304 tokens (8304 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}